In [17]:
from pyspark.sql import SparkSession #import library
from pyspark.sql.functions import avg, col, desc

In [2]:
spark = SparkSession.builder\
    .appName("Student Scores Project")\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/19 17:51:33 WARN Utils: Your hostname, DESKTOP-H7B5I93, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/19 17:51:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/19 17:51:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv("students.csv", header=True, inferSchema=True) #loading csv

In [5]:
df_clean = df.dropna(subset=["Country"])

In [6]:
avg_maths = df_clean.select(avg("Math")).first()[0]
avg_english = df_clean.select(avg("English")).first()[0]

In [7]:
df_filled= df_clean.fillna({
    "Math": avg_maths,
    "English": avg_english
})

In [8]:
print (df_filled)

DataFrame[Name: string, Math: double, English: double, Country: string]


In [22]:
df_final = df_filled.withColumn("Total", col("Math") + col("English")) \
                    .withColumn("Average", (col("Math") + col("English")) / 2)
df_final.show()

+--------+-----+-------+------------+-----+-------+
|    Name| Math|English|     Country|Total|Average|
+--------+-----+-------+------------+-----+-------+
|  Yvette| 85.0|   83.0|       Ghana|168.0|   84.0|
|    John| 86.9|   79.0|    Ethiopia|165.9|  82.95|
|Patricia| 86.9|   61.0|South Africa|147.9|  73.95|
|  Alicia| 86.9|   79.0|South Africa|165.9|  82.95|
|    Jill| 86.9|   69.0|       Ghana|155.9|  77.95|
|    Fred| 86.9|   60.0|    Ethiopia|146.9|  73.45|
|    Alex| 86.9|   97.0|       Ghana|183.9|  91.95|
| Christy|100.0|   70.0|     Nigeria|170.0|   85.0|
|  Angela| 86.9|   79.0|       Ghana|165.9|  82.95|
| Timothy| 75.0|   82.0|South Africa|157.0|   78.5|
|  George| 86.9|   92.0|       Ghana|178.9|  89.45|
| Brendan| 86.9|   79.0|    Ethiopia|165.9|  82.95|
|Nicholas| 88.0|   97.0|    Ethiopia|185.0|   92.5|
|    Lisa| 95.0|   79.0|       Ghana|174.0|   87.0|
|    Jose| 99.0|   79.0|    Ethiopia|178.0|   89.0|
|  Kelsey| 86.9|   97.0|       Ghana|183.9|  91.95|
| Douglas| 5

In [23]:
df_avg_country = df_final.groupBy("Country").avg("Average")
df_avg_country.show()

+------------+-----------------+
|     Country|     avg(Average)|
+------------+-----------------+
|       Ghana|         86.65625|
|     Nigeria|           78.725|
|    Ethiopia|84.53571428571429|
|       Kenya|             73.0|
|South Africa|            79.85|
+------------+-----------------+



In [19]:
df_top_students = df_final.orderBy(desc("Total")).limit(3)
df_top_students.show()

+--------+----+-------+--------+-----+-------+
|    Name|Math|English| Country|Total|Average|
+--------+----+-------+--------+-----+-------+
|Nicholas|88.0|   97.0|Ethiopia|185.0|   92.5|
|    Alex|86.9|   97.0|   Ghana|183.9|  91.95|
|  Kelsey|86.9|   97.0|   Ghana|183.9|  91.95|
+--------+----+-------+--------+-----+-------+



In [20]:
df_final.write.mode("overwrite").csv("output/cleaned_students", header=True)